In [1]:
from pyspark.sql import *
spark=SparkSession.builder.appName('s18').getOrCreate()

In [2]:
df=spark.read.csv('/FileStore/tables/provider_level_measure_rates_for_ahrq_patient_safety_indicator_11_psi_11-9fa94.csv', inferSchema=True, header=True)

In [3]:
df.na.fill(5).dropDuplicates().show(5,truncate=True)

+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
HOSP_ID|ADM_DISC| RATE|INTERVAL_LOWER_LIMIT|INTERVAL_HIGHER_LIMIT|START_QUARTER|END_QUARTER| START_DATE| END_DATE|
+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
 30122| 428|15.02| 9.47| 20.58| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 50169| 513|10.06| 5.03| 15.09| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 110146| 104|11.58| 4.93| 18.22| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 140013| 895| 5.46| 0.9| 10.02| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 220176| 628| 7.39| 2.39| 12.39| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
only showing top 5 rows

In [4]:
df1=df.select('rate').filter(df['Hosp_id']>5000).na.fill(5)

In [5]:
df1.show(5)

+-----+
 rate|
+-----+
 6.5|
12.91|
 5.0|
10.98|
 5.0|
+-----+
only showing top 5 rows

In [6]:
df2=df.groupBy(df.ADM_DISC)

In [7]:
df3=df2.avg('RATE').alias('avg')

In [8]:
df3.show()

+--------+------------------+
ADM_DISC| avg(RATE)|
+--------+------------------+
 148|10.856666666666667|
 471|11.280000000000001|
 496| 9.850000000000001|
 1990| 11.58|
 858| 11.78|
 897| 8.16|
 623|13.583333333333334|
 1896| 14.24|
 1084| 13.205|
 540| 15.33|
 243|10.079999999999998|
 1460| 10.41|
 392| 12.6|
 737| 15.85|
 1025| 8.19|
 1395| 15.73|
 1127| 9.62|
 31| 12.5275|
 2572| 10.94|
 1303| 9.46|
+--------+------------------+
only showing top 20 rows

In [9]:
df4=df2.max('INTERVAL_LOWER_LIMIT')
df4.show()

+--------+-------------------------+
ADM_DISC|max(INTERVAL_LOWER_LIMIT)|
+--------+-------------------------+
 148| 4.74|
 471| 8.84|
 496| 5.23|
 1990| 8.26|
 858| 7.66|
 897| 5.38|
 623| 10.57|
 1896| 10.91|
 1084| 9.43|
 540| 10.94|
 243| 5.06|
 1460| 7.07|
 392| 11.37|
 737| 11.15|
 1025| 4.0|
 1395| 11.8|
 1127| 6.02|
 31| 8.81|
 2572| 7.9|
 1303| 6.26|
+--------+-------------------------+
only showing top 20 rows

In [10]:
df5=df.createOrReplaceTempView('data')

In [11]:
df6=spark.sql('select max(rate) as max,min(rate) as min from data')

In [12]:
df6.show()

+-----+----+
 max| min|
+-----+----+
36.18|3.49|
+-----+----+

In [13]:
df7=spark.sql('select count(*) as count from data')

In [14]:
-df7.show()

+-----+
count|
+-----+
 3410|
+-----+

In [15]:
df8=spark.sql('select rate,adm_disc,count(*) as count from data group by rate,adm_disc')

--------------------------------------------------------------------------- 
 ParseException Traceback (most recent call last)
 <command-3668116202313221> in <module> () 
 ----> 1 df8 = spark . sql ( 'select rate,adm_disc,count(*) as count from data Order By rate desc group by rate,adm_disc' ) 

 /databricks/spark/python/pyspark/sql/session.py in sql (self, sqlQuery) 
 746 [ Row ( f1 = 1 , f2 = u'row1' ) , Row ( f1 = 2 , f2 = u'row2' ) , Row ( f1 = 3 , f2 = u'row3' ) ] 
 747 """
 --> 748 return DataFrame ( self . _jsparkSession . sql ( sqlQuery ) , self . _wrapped ) 
 749 
 750 @ since ( 2.0 ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 72 if s . startswith ( 'org.apache.spark.sql.catalyst.parser.ParseException: ' ) : 
 ---> 73 raise ParseException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 74 if s . startswith ( 'org.apache.spark.sql.streaming.StreamingQueryException: ' ) : 
 75 raise StreamingQueryException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 ParseException : "\nmismatched input 'group' expecting <EOF>(line 1, pos 68)\n\n== SQL ==\nselect rate,adm_disc,count(*) as count from data Order By rate desc group by rate,adm_disc\n--------------------------------------------------------------------^^^\n"

In [16]:
df8.show()

+-----+--------+-----+
 rate|adm_disc|count|
+-----+--------+-----+
11.46| 247| 1|
10.71| 40| 1|
10.05| 640| 1|
11.44| 503| 1|
10.49| 236| 1|
15.85| 737| 1|
11.01| 384| 1|
 9.69| 247| 1|
10.43| 205| 1|
 8.16| 1063| 1|
10.98| 72| 1|
10.55| 95| 1|
 11.4| 109| 1|
19.34| 1137| 1|
13.04| 2047| 1|
13.03| 75| 1|
 9.45| 1337| 1|
 8.6| 741| 1|
10.26| 656| 1|
 11.0| 90| 1|
+-----+--------+-----+
only showing top 20 rows

In [17]:
df.filter(df['hosp_id']>5000).select('rate', 'adm_disc').show()

+-----+--------+
 rate|adm_disc|
+-----+--------+
 6.5| 1901|
12.91| 269|
 null| null|
10.98| 56|
 null| null|
 null| null|
11.32| 48|
11.92| 1026|
11.46| 196|
 null| null|
11.59| 719|
11.67| 27|
12.36| 502|
 10.4| 101|
 null| null|
11.44| 1569|
14.61| 1258|
 null| null|
 null| null|
 9.97| 1606|
+-----+--------+
only showing top 20 rows

In [18]:
df9=spark.sql('select * from data Order BY rate desc')

In [19]:
df9.show()

+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
HOSP_ID|ADM_DISC| RATE|INTERVAL_LOWER_LIMIT|INTERVAL_HIGHER_LIMIT|START_QUARTER|END_QUARTER| START_DATE| END_DATE|
+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
 670025| 379|36.18| 31.32| 41.04| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 50100| 1365|32.86| 29.13| 36.6| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 490112| 2353|32.48| 29.46| 35.51| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 110035| 2957|31.34| 28.68| 34.0| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 440048| 3160|30.15| 27.74| 32.56| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 160083| 2984|29.81| 27.17| 32.44| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 490136| 1135|29.34| 25.07| 33.62| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 50696| 2230|29.13| 26.77| 31.48| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 110143| 1012|28.75| 24.64| 32.86| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 260096| 2311|28.64| 25.4| 31.87| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 450193| 1979|28.31| 25.38| 31.25| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 50224| 1214|28.22| 25.27| 31.16| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 290007| 714|27.73| 23.2| 32.26| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 230141| 1820|27.11| 24.12| 30.1| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 360059| 698|26.37| 21.62| 31.12| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 230207| 267|25.92| 20.28| 31.56| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 360074| 413|25.71| 20.71| 30.7| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 310002| 518|25.18| 20.34| 30.01| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 100128| 3420| 25.0| 22.94| 27.07| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 440133| 2114|24.82| 21.81| 27.82| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
only showing top 20 rows

In [20]:
df10=spark.sql('select avg(rate) as rate,max(adm_disc) as max from data where hosp_id > 5000')


In [21]:
df10.show()

+------------------+-----+
 rate| max|
+------------------+-----+
11.776605088881134|11180|
+------------------+-----+

In [22]:
df11=spark.sql('select hosp_id,rate,count(hosp_id) from data group by hosp_id,rate')

In [23]:
df11.show(5)

+-------+-----+--------------+
hosp_id| rate|count(hosp_id)|
+-------+-----+--------------+
 50435|10.18| 1|
 50757| 9.34| 1|
 70004|10.26| 1|
 100014|13.17| 1|
 130002| 8.41| 1|
+-------+-----+--------------+
only showing top 5 rows

In [24]:
df12=spark.sql('select rate from data group by rate')


In [25]:
df12.show()

+-----+
 rate|
+-----+
10.65|
 8.51|
 14.9|
 13.4|
12.32|
16.75|
 15.4|
 6.96|
23.06|
 9.55|
 5.86|
 9.72|
 8.73|
11.53|
14.18|
14.75|
 5.05|
 8.0|
 9.49|
10.82|
+-----+
only showing top 20 rows

In [26]:
df.show()

+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
HOSP_ID|ADM_DISC| RATE|INTERVAL_LOWER_LIMIT|INTERVAL_HIGHER_LIMIT|START_QUARTER|END_QUARTER| START_DATE| END_DATE|
+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
 10001| 1901| 6.5| 4.01| 8.98| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10005| 269|12.91| 7.32| 18.5| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10006| null| null| null| null| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10007| 56|10.98| 4.06| 17.9| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10008| null| null| null| null| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10009| null| null| null| null| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10010| 48|11.32| 4.29| 18.34| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10011| 1026|11.92| 7.99| 15.85| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10012| 196|11.46| 5.23| 17.69| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10015| null| null| null| null| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10016| 719|11.59| 7.67| 15.51| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10018| 27|11.67| 4.54| 18.8| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10019| 502|12.36| 7.39| 17.33| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10021| 101| 10.4| 4.16| 16.65| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10022| null| null| null| null| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10023| 1569|11.44| 8.11| 14.76| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10024| 1258|14.61| 10.93| 18.29| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10025| null| null| null| null| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10027| null| null| null| null| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10029| 1606| 9.97| 6.47| 13.47| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
only showing top 20 rows

In [27]:
df12=spark.sql('select max(hosp_id) as max,min(hosp_id) as min from data')

In [28]:
df12.show()

+------+-----+
 max| min|
+------+-----+
670085|10001|
+------+-----+

In [29]:
df13=spark.sql('select rate,count(*) from data group by rate having count(*)>1 ')

In [30]:
df13.show()

+-----+--------+
 rate|count(1)|
+-----+--------+
10.65| 6|
 8.51| 3|
 14.9| 2|
 13.4| 4|
12.32| 4|
16.75| 3|
 15.4| 2|
 9.55| 3|
 9.72| 6|
 8.73| 6|
11.53| 5|
14.18| 2|
14.75| 3|
 8.0| 4|
 9.49| 3|
10.82| 7|
14.68| 2|
 10.2| 2|
 8.43| 3|
 9.41| 4|
+-----+--------+
only showing top 20 rows

In [31]:
df14=spark.sql('select adm_disc from data where hosp_id = (select min(hosp_id) from data)')  

In [32]:
df14.show()

+--------+
adm_disc|
+--------+
 1901|
+--------+

In [33]:
df15=spark.sql('select distinct adm_disc from data where hosp_id IN(select hosp_id from data)')

In [34]:
df15.show()

+--------+
adm_disc|
+--------+
 148|
 471|
 496|
 1990|
 858|
 897|
 623|
 1896|
 1084|
 540|
 243|
 1460|
 392|
 737|
 1025|
 1395|
 1127|
 31|
 2572|
 1303|
+--------+
only showing top 20 rows

In [35]:
df16=spark.sql('select hosp_id, max(rate) from data group by hosp_id')

In [36]:
df17=spark.sql('select * from data where (adm_disc, rate) IN (select adm_disc, max(rate) from data group by adm_disc)')

In [37]:
df17.show()

+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
HOSP_ID|ADM_DISC| RATE|INTERVAL_LOWER_LIMIT|INTERVAL_HIGHER_LIMIT|START_QUARTER|END_QUARTER| START_DATE| END_DATE|
+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
 10001| 1901| 6.5| 4.01| 8.98| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10011| 1026|11.92| 7.99| 15.85| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10019| 502|12.36| 7.39| 17.33| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10023| 1569|11.44| 8.11| 14.76| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10024| 1258|14.61| 10.93| 18.29| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10029| 1606| 9.97| 6.47| 13.47| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10033| 4355|12.79| 10.88| 14.7| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10039| 4640| 9.87| 7.49| 12.25| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10040| 1199|10.65| 6.67| 14.63| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10049| 351| 9.69| 3.6| 15.79| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10055| 1857| 8.73| 5.45| 12.02| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10056| 3410|10.57| 7.49| 13.66| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10079| 181|16.41| 10.19| 22.64| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10085| 454|13.27| 8.78| 17.77| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10087| 198|16.65| 11.23| 22.08| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10090| 1251|10.85| 7.0| 14.7| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10100| 887| 7.86| 2.95| 12.78| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10113| 1990|11.58| 8.26| 14.91| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10114| 372|14.35| 9.29| 19.42| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
 10131| 1429|15.31| 11.18| 19.44| 3Q2011| 2Q2013|2011-07-01 00:00:00|2013-06-30 00:00:00|
+-------+--------+-----+--------------------+---------------------+-------------+-----------+-------------------+-------------------+
only showing top 20 rows